In [1]:
#returns the html content of given page
def render(source_html):
    #Fully render HTML, JavaScript and all
    import sys
    from PyQt5.QtCore import QEventLoop
    from PyQt5.QtWidgets import QApplication
    from PyQt5.QtWebEngineWidgets import QWebEngineView

    class Render(QWebEngineView):
        def __init__(self, html):
            self.html = None
            self.app = QApplication(sys.argv)
            QWebEngineView.__init__(self)
            self.loadFinished.connect(self._loadFinished)
            self.setHtml(html)
            while self.html is None:
                self.app.processEvents(QEventLoop.ExcludeUserInputEvents | QEventLoop.ExcludeSocketNotifiers | QEventLoop.WaitForMoreEvents)
            self.app.quit()

        def _callable(self, data):
            self.html = data

        def _loadFinished(self, result):
            self.page().toHtml(self._callable)

    return Render(source_html).html

#Extract all the content from td tags and check for rows with no nulls values and len(cols)=count
def extract_data(souptable,col_count): 
    arr=[]    
    for row in souptable.find_all('tr')[1:]:
        cols = row.find_all('td')
        if len(cols) == col_count:
            arr.append([])
            for c in range(len(cols)):
                value=cols[c].find(text=True)
                arr[-1].append( value )
    return arr

import requests
import pandas as pd
from bs4 import BeautifulSoup
#Open the options URL
opts_html = requests.get("https://finance.yahoo.com/quote/AMZN/options?p=AMZN").text
rendered_opts_html = render(opts_html)
#Genrate the text and parse html and find 'table' tag
soup_o = BeautifulSoup(rendered_opts_html, 'html.parser')
opts_souptable = soup_o.find('table')

opt_columns = [ "Contract Name",
           "Last_Trade_Date",
           "Strike",
           "Last Price",
           "Bid",
           "Ask",
           "Change",
           "Percentage_Change",
           "Volume",
           "Open Interest",
           "Implied Volatility"
          ]
#Extract data method return the values scraped from the yahoo page
opts_data=extract_data(opts_souptable,10)

#create a dataframe using the data and add column names
opts_df = pd.DataFrame(opts_data, columns=opt_columns )

#print the sample data that is generated
print(opts_df.head())

#Write the Options data to opts.csv
opts_df.to_csv("opts.csv",index=False)




AttributeError: 'NoneType' object has no attribute 'find_all'